# E.D.A.: CORRELATION BETWEEN  UNEMPLOYMENT & SUICIDE RATES IN 43 COUNTRIES FROM 2000 TO 2016  
By Daniel Del Valle González  2020-2021

## STEP 1 - DATA CLEANING & NORMALISING
## STEP 2 - DATA ANALISYS & TENDENCIES. TOP & BOTTOM COUNTRIES IN SUICIDE & UNEMPLOYMENT RATES
## STEP 3 - GRAPHIC VISUALISATION OF EACH DATAFRAME TENDENCIES

## STEP 1 - DATA CLEANING & NORMALISING

### We got 2 csv from:   
1 - https://www.kaggle.com/szamil/who-suicide-statistics csv for suicides from 2000.  
2 - https://stats.oecd.org/Index.aspx?QueryId=64198# csv for unemployment rates. I selected and added some countries that by defect were not included.  

We create 2 DataFrames from them:

In [1]:
import pandas as pd
import numpy as np
from utils.folders_tb import *
from utils.mining_data_tb import *
from utils.visualization_tb import *

## Both had to be cleaned and "synchronised" in many ways:

In [2]:
suicide = pd.read_csv("C:\\DATA_SCIENCE\\PROYECTO\\documentation\\who_suicide_statistics.csv")
unemployment = pd.read_csv("C:\\DATA_SCIENCE\\PROYECTO\\documentation\\unemployment_all_ratio.csv")

In [3]:
suic, unemp = intersector(df1=suicide, df2=unemployment, col1="country", col2="Country")      #creating dataframes with data about only their shared countries

### Unemployment CSV also included "employment rate" and other values (columns). We only select those regarding to "unemployment rate"

In [4]:
unemp = only_desired(df=unemp, col1="Series", desired="Unemployment rate")  #selecting onlye "unemployment" stadistic values

### Unemployment DataFrame had some groups of countries that did not appear on the Suicides Rates DataFrame, so are discarded.

In [5]:
unemp = str_discarder(unemp, "Country", "OECD")                              #discarding groups of countries(no info in the other dataframe)
unemp = str_discarder(unemp, "Country", "Euro")

### Columns and values from both dataframes are to be renamed with similar names to allow easy contrast

In [6]:
unemp = column_renamer(unemp, ['SEX', 'Value', 'Time'], ['Gender', 'Unemploy_Rate', 'Year'])
suic = column_renamer(suic, ['sex'], ['gender'])

In [7]:
unemp = unemp[['Country', 'Gender', 'Age', 'Year', 'Unemploy_Rate']]      #selection only useful columns in uneployment dataframe

In [8]:
column_lower(suic)                                                        #normalising columns names
column_lower(unemp)                                                                    

In [9]:
value_renamer(unemp, "gender", "MEN", "male")                             #normalising "gender" column formats   
value_renamer(unemp, "gender", "WOMEN", "female")

In [10]:
value_discarder(unemp, "gender", "MW") #discarding MW values as they are ambiguous, unaccurate summatories"""

In [11]:
str_replacer(unemp, "age", " to ", "-")

In [12]:
str_cleaner(suic, "age", "years ")

### Dataframes had data in different years ranges, so we had to cut them to fit:

In [13]:
unemp = unemp[unemp['year'] <= 2016]
suic = suic[suic['year'] >= 2000]

### On the "age" columns the ranges where different (fewer and broader in Suicides Df, narrower in Unemployment Df. Synchronisation was not an easy task)

In [14]:
unemp = unemp[(unemp['age'] == '15-24') | (unemp['age'] == '25-34') |(unemp['age'] == '35-44') |(unemp['age'] == '45-54') | (unemp['age'] == '55-64') |(unemp['age'] == '65-69') | (unemp['age'] == '70-74')]


In [15]:
unemp = unemp[(unemp['age'] == '15-24') | (unemp['age'] == '25-34') |(unemp['age'] == '35-44') |(unemp['age'] == '45-54') | (unemp['age'] == '55-64') |(unemp['age'] == '65-69') | (unemp['age'] == '70-74')]                                                                                               #discarding duplicated age ranges(some are include inside others) 

unemp.loc[(unemp["age"] == '35-44') | (unemp["age"] == '45-54'), 'age'] = '35-54'                                  #merging smaller ranges into a bigg one (and common with my other csv)
unemp.loc[(unemp["age"] == '55-64') | (unemp["age"] == '65-69') | (unemp["age"] == '70-74'), 'age'] = "55-74"

## STEP 2 - DATA ANALISYS & TENDENCIES. TOP & BOTTOM COUNTRIES IN SUICIDE & UNEMPLOYMENT RATES

### Suicide is stadistically more common visualised in "suicides rate per 100.000 people", so we create a column regarding that measure:

In [16]:
add_ratio(suic, "suic_100k", "suicides_no", "population", 100000, 2)   #creating a column with desired value; just 2 decimals is enough

### Then we create some sub-dataframes with info centered in Age: 

In [20]:
aggregation_functions = {'suic_100k': 'mean'}                                       #creating agg functions to make it faster when grouping tables
aggregation_functions2 = {'unemploy_rate': 'mean'}

In [21]:
suic_ages_mean = suic.groupby(suic['age']).aggregate(aggregation_functions).sort_values("suic_100k",ascending=False).round(2)

In [22]:
unemp_ages_mean = unemp.groupby('age').aggregate(aggregation_functions2).sort_values("unemploy_rate", ascending=False).round(2)

### Also we create some sub-dataframes with info centered in distribution per Country:

In [23]:
suic_countries_mean = suic.groupby('country').aggregate(aggregation_functions).sort_values("suic_100k",ascending=True).round(2)

In [24]:
unemp_countries_mean = unemp.groupby('country').aggregate(aggregation_functions2).sort_values("unemploy_rate", ascending=False).round(2)

In [25]:
most_per_100k = suic_countries_mean.sort_values("suic_100k", ascending=False).round(2).head()
least_per_100k = suic_countries_mean.sort_values("suic_100k", ascending=False).round(2).tail()

In [26]:
most_of_all = suic.groupby("country").sum().sort_values("suicides_no", ascending=False).head()                #top and bottom countries for absolute number of suicides
least_of_all = suic.groupby("country").sum().sort_values("suicides_no", ascending=False).tail()

In [27]:
most_unemp = unemp_countries_mean.sort_values("unemploy_rate", ascending=False).round(2).head(5)               #top and bottom countries for unemployment rate
least_unemp = unemp_countries_mean.sort_values("unemploy_rate", ascending=False).round(2).tail(5)

### Suicide countries of Interest:

In [28]:
suic_pivot = pd.pivot_table(suic, index = ['country', 'year'], values = ['suicides_no', 'suic_100k']).round(2)

In [29]:
lt_su = suic_pivot.loc["Lithuania"]
lt_su

,suic_100k,suicides_no
year,,
2000,47.65,135.92
2001,47.42,127.75
2002,46.30,129.25
2003,44.21,121.25
2004,41.95,115.00
2005,40.48,109.92
2006,33.38,87.42
2007,33.90,85.42
2008,35.84,92.58


In [30]:
ru_su = suic_pivot.loc["Russian Federation"]
ru_su

,suic_100k,suicides_no
year,,
2000,42.04,4718.25
2001,42.05,4746.50
2002,40.71,4585.33
2003,39.08,4287.08
2004,37.30,4091.33
2005,35.03,3816.83
2006,33.44,3551.17
2007,32.25,3429.08
2008,30.57,3184.25


In [31]:
sa_su = suic_pivot.loc["South Africa"]
sa_su

,suic_100k,suicides_no
year,,
2000,0.80,22.25
2001,1.04,32.00
2002,0.58,19.67
2003,0.77,21.33
2004,1.05,31.92
2005,1.27,38.17
2006,1.49,42.83
2007,1.01,34.75
2008,1.18,36.75


### Unemployment countries of Interest:

In [32]:
unemp_pivot_mean_gndr = pd.pivot_table(unemp, index = ['country', 'year', 'gender'], values = ['unemploy_rate']).round(2)
unemp_pivot_mean_gndr

unemploy_rate
country        year gender               
Australia      2000 female           4.99
                    male             5.87
               2001 female           5.35
                    male             6.40
               2002 female           5.16
...                                   ...
United Kingdom 2014 male             6.75
               2015 female           4.97
                    male             5.58
               2016 female           4.71
                    male             5.17

[1448 rows x 1 columns]

In [33]:
sp_un = unemp_pivot_mean_gndr.loc["Spain"]                              #unemployment evolution in Spain per gender : tendency is to equal
sp_un

unemploy_rate
year gender               
2000 female          15.31
     male             8.64
2001 female          11.21
     male             6.74
2002 female          11.88
     male             6.83
2003 female          12.13
     male             7.04
2004 female          11.34
     male             6.84
2005 female          10.02
     male             6.17
2006 female           9.15
     male             5.49
2007 female           8.84
     male             5.50
2008 female          10.85
     male             8.31
2009 female          14.87
     male            14.61
2010 female          15.99
     male            16.75
2011 female          17.92
     male            18.05
2012 female          21.23
     male            21.03
2013 female          22.90
     male            22.53
2014 female          22.12
     male            21.33
2015 female          20.03
     male            19.00
2016 female          19.16
     male            17.09

In [34]:
unemp_pivot_mean = pd.pivot_table(unemp, index = ['country', 'year'], values = ['unemploy_rate']).round(2)
unemp_pivot_mean

unemploy_rate
country        year               
Australia      2000           5.43
               2001           5.88
               2002           5.51
               2003           5.15
               2004           4.78
...                            ...
United Kingdom 2012           7.72
               2013           7.41
               2014           6.29
               2015           5.27
               2016           4.94

[724 rows x 1 columns]

In [35]:
nw_un = unemp_pivot_mean.loc["Norway"]
nw_un

,unemploy_rate
year,
2000,3.57
2001,3.53
2002,3.15
2003,3.58
2004,3.46
2005,3.67
2006,2.94
2007,2.21
2008,2.12


In [36]:
sa_un = unemp_pivot_mean.loc["South Africa"]
sa_un

,unemploy_rate
year,
2001,22.99
2002,24.65
2003,24.74
2004,22.58
2005,21.89
2006,20.64
2007,20.59
2008,18.43
2009,19.37


## STEP 3 - GRAPHIC VISUALISATION OF EACH DATAFRAME TENDENCIES

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import psutil
import plotly.io as pio
from Unemployment import *
from Suicide import *

ModuleNotFoundError: No module named 'Unemployment'